## Glass Transition Temperature Project Manager


In [1]:
import os
import pathlib as pl
import shutil
import pandas as pd
from ipywidgets import Button, Text, HBox
from IPython.display import clear_output
import qgrid
qgrid.enable()

In [2]:
add_button = Button (
                description='Add Project',
                icon='plus',
                tooltip='Add a New Project',
                button_style='success',
                disabled=True
            )
del_button = Button (
                description='Delete Project',
                icon='minus',
                tooltip='Remove Project and its Files',
                button_style='danger',
                disabled=True
            )
style = {'description_width': 'initial'}
name_field = Text(description='Project Name', style=style)

In [3]:
# Copy projects from apps directory to ~/TGNB
sdir = os.path.dirname(os.path.realpath('launcher.ipynb'))
pfile = os.path.join(sdir, 'projects.json')
pdir =  os.path.join(sdir, 'projects')
if not os.path.exists('projects.json'):
    shutil.copy(pfile, '.')
    shutil.copytree(pdir, 'projects')

In [4]:
projects_file = 'projects.json'
df = None
if pl.Path(projects_file).exists():
    df = pd.read_json(projects_file)

def create_project(_):
    global df
    name = name_field.value
    path = pl.Path('projects') / name
    tool_destination = path / 'Tg_{}.ipynb'.format(name)
    # tool_link = '<a href="{}">Installed Tool</a>'.format(tool_destination)
    # changed below to open a new tab
    tool_link = '<a href="{}" target="_blank">Launch</a>'.format(tool_destination)
    
    path.mkdir(parents=True)
    
    # need to copy an installed notebookfrom /apps/...
    srcdir = os.path.join(sdir, 'bin')
    nb = 'Tg.ipynb'
    src = pl.Path(srcdir) / nb
    shutil.copy(src, tool_destination)
    project_info = {'project': name,
                    'directory': str(path),
                    'tool': str(tool_link)}
    if df is None:
        df = pd.DataFrame([project_info])
    else:
        df = df.append(pd.DataFrame([project_info]), ignore_index=True)
    df.to_json(projects_file)
    name_field.value = ''
    show_projects()

    
def del_project(_):
    global df
    name = name_field.value
    loc = df.loc[df['project'] == name].index[0]
    df = df.drop(loc)
    df = df.reset_index(drop=True)
    path = pl.Path('projects') / name
    shutil.rmtree(path)
    df.to_json(projects_file)
    name_field.value = ''
    show_projects()


In [5]:
# Callback when the name_field changes
# Check the path and enable/disable the add button
def project_name(change):
    name = change['new']
    path = pl.Path('projects') / name
    if path.exists():
        add_button.disabled = True
        del_button.disabled = False
    else:
        add_button.disabled = False
        del_button.disabled = True

In [6]:
def select_cb(event, w):
    ind = event['new'][0]
    name = df.loc[ind, 'project']
    name_field.value = name
    

In [7]:
def show_projects():
    global df
    clear_output()
    
    if df is None or len(df) == 0:
        display('No projects exist. Make a project first.')
    else:
        df2 = df.set_index('project')
        display(qgrid.show_grid(df2, grid_options={'editable': False}))
    display(HBox([name_field, add_button, del_button]))

show_projects()
qgrid.on('selection_changed', select_cb)
name_field.observe(project_name, names='value')
add_button.on_click(create_project)
del_button.on_click(del_project)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': False, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='03f7ef6e-642b-427c-bf9d-ff76408ccd4b', precision=5)

HBox(children=(Text(value='', description='Project Name', style=DescriptionStyle(description_width='initial')), Button(button_style='success', description='Add Project', disabled=True, icon='plus', style=ButtonStyle(), tooltip='Add a New Project'), Button(button_style='danger', description='Delete Project', disabled=True, icon='minus', style=ButtonStyle(), tooltip='Remove Project and its Files')))